# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,west island,-12.1568,96.8225,23.99,78,75,9.77,CC,1690499668
1,1,grytviken,-54.2811,-36.5092,-2.28,84,100,4.61,GS,1690499668
2,2,kloulklubed,7.0419,134.2556,25.28,76,100,11.76,PW,1690499668
3,3,atafu village,-8.5421,-172.5159,28.07,78,27,7.35,TK,1690499669
4,4,yellowknife,62.4560,-114.3525,24.29,31,75,5.14,CA,1690499669


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
"Lng","Lat",geo=True, tiles="OSM",
size = "Humidity",color = "City")

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Define the criteria for filtering the cities
humidity_criteria = 50
temperature_criteria = 20

# Filter the cities that meet the criteria using pandas query method
filtered_cities_df = city_data_df[(city_data_df["Humidity"] < humidity_criteria) & (city_data_df["Max Temp"] > temperature_criteria)]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
print(filtered_cities_df)


     City_ID                    City      Lat       Lng  Max Temp  Humidity  \
4          4             yellowknife  62.4560 -114.3525     24.29        31   
5          5  happy valley-goose bay  53.3168  -60.3315     21.17        43   
51        51              whitehorse  60.7161 -135.0538     25.01        49   
58        58           belle fourche  44.6714 -103.8522     26.73        18   
65        65                  at taj  24.2049   23.2857     30.72        12   
..       ...                     ...      ...       ...       ...       ...   
535      535                   seydi  39.4816   62.9137     29.76        32   
546      546             sao vicente -23.9631  -46.3919     23.10        26   
550      550             al kharijah  25.4514   30.5464     31.61        16   
558      558               goianesia -15.3175  -49.1175     23.52        30   
560      560               oriximina  -1.7656  -55.8661     29.31        47   

     Cloudiness  Wind Speed Country        Date  
4

### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Create a new DataFrame called hotel_df by copying the city_data_df
hotel_df = city_data_df.copy()

# Select only the columns needed in hotel_df (city, country, coordinates, and humidity)
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

In [6]:
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,west island,CC,-12.1568,96.8225,78,
1,grytviken,GS,-54.2811,-36.5092,84,
2,kloulklubed,PW,7.0419,134.2556,76,
3,atafu village,TK,-8.5421,-172.5159,78,
4,yellowknife,CA,62.4560,-114.3525,31,
...,...,...,...,...,...,...
562,sola,VU,-13.8833,167.5500,77,
563,ouesso,CG,1.6136,16.0517,80,
564,n'zeto,AO,-7.2312,12.8666,89,
565,vung tau,VN,10.3460,107.0843,87,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}" 
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_name = name_address["features"][0]["properties"]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
    except (KeyError, IndexError)as e:
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
               
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
print (hotel_df)

Starting hotel search
west island - nearest hotel: Cocos Village Bungalows
grytviken - nearest hotel: No hotel found
kloulklubed - nearest hotel: The Adventures Inn
atafu village - nearest hotel: No hotel found
yellowknife - nearest hotel: No hotel found
happy valley-goose bay - nearest hotel: Hotel North Two
enewetak - nearest hotel: No hotel found
bethel - nearest hotel: Hampton Inn Danbury
puerto natales - nearest hotel: Dorotea Patagonia Hostel
raahe - nearest hotel: Aparthotel Forenom
bure - nearest hotel: belay zeleke hotel
metkovic - nearest hotel: Hotel MB
adamstown - nearest hotel: No hotel found
port-aux-francais - nearest hotel: Keravel
vilyuchinsk - nearest hotel: Победа
sitangkai - nearest hotel: No hotel found
cesky krumlov - nearest hotel: Hotel Grand
thompson - nearest hotel: Thompson Inn
papao - nearest hotel: Hiti Moana Villa Lodge
leh - nearest hotel: Kang Lha Chen
cabugao - nearest hotel: No hotel found
mugia - nearest hotel: Solpor do Coído
neiafu - nearest hotel: 

In [8]:
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,west island,CC,-12.1568,96.8225,78,Cocos Village Bungalows
1,grytviken,GS,-54.2811,-36.5092,84,No hotel found
2,kloulklubed,PW,7.0419,134.2556,76,The Adventures Inn
3,atafu village,TK,-8.5421,-172.5159,78,No hotel found
4,yellowknife,CA,62.4560,-114.3525,31,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display
# Combine the hotel name and country information
# hotel_df["Hover Info"] = hotel_df["Hotel Name"] + ", " + hotel_df["Country"]

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", "Lat", geo=True, tiles="OSM",
    size="Humidity", color="City", hover_cols=["Hotel Name","Country"]
)

# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)